In [5]:
import numpy as np
import matplotlib.pyplot as plt
import deepxde as dde 
from analytical import sound_hard_circle, sound_hard_circle_deepxde


#dde.config.set_default_float('float64')
#dde.config.set_default_float('float64')
dde.config.set_default_float('float32')


'''                        Problem parameters                               '''
k0 = 2           # wavenumber
# 5 is fine!
# 8 is more or less ok. ¿Add more points?
# Added more points, but not really ok with just 20.000 iterations + BFGS
# Change the learning rate? 
# Avec8 et 15 points, learning rate = 0.05, n'arrive pas a apprendre l'interieur...

wave_len = np.pi / k0  # wavelength

dim_x = 3 * np.pi

R = np.pi / 2.
n_wave = 5

# The mesh element size is h_elem
h_elem = wave_len / n_wave

nx = int(dim_x / h_elem)
print(h_elem, 'h_elem')
print( nx, 'nx')
print(nx **2, 'nx**2')

Set the default float type to float32
0.3141592653589793 h_elem
30 nx
900 nx**2


In [ ]:
#geom = dde.geometry.Rectangle([-dim_x/2., -dim_x/2.], [dim_x/2., dim_x/2.])

outer = dde.geometry.Rectangle([-dim_x/2., -dim_x/2.], [dim_x/2., dim_x/2.])
inner = dde.geometry.Disk([0,0], R)

geom = dde.geometry.CSGDifference(outer, inner)

def pde(x, y):
    y0, y1 = y[:, 0:1], y[:, 1:2]
    
    y0_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    y0_yy = dde.grad.hessian(y, x, component=0, i=1, j=1)

    y1_xx = dde.grad.hessian(y, x,component=1, i=0, j=0)
    y1_yy = dde.grad.hessian(y, x,component=1, i=1, j=1)

    return [-y0_xx - y0_yy - k0 ** 2 * y0,
            -y1_xx - y1_yy - k0 ** 2 * y1]

def sol(x):
    from scipy.special import jv, hankel1
    result = sound_hard_circle_deepxde(k0, R, x).reshape((x.shape[0],1))
    real = np.real(result)
    imag = np.imag(result)
    return np.hstack((real, imag))

def boundary(_, on_boundary):
    return on_boundary

def boundary_outer(_, on_boundary):
    return on_boundary and outer.on_boundary(_)

def boundary_inner(_, on_boundary):
    return on_boundary and inner.on_boundary(_)

def func0_inner(x):
    #result = np.exp(1j * k0 * x[:, 0:1])
    normal = -inner.boundary_normal(x)
    g = 1j * k0 * np.exp(1j * k0 * x[:, 0:1]) * normal[:, 0:1]
    return np.real(-g)

def func1_inner(x):
    #result = np.exp(1j * k0 * x[:, 0:1])
    normal = -inner.boundary_normal(x)
    g = 1j * k0 * np.exp(1j * k0 * x[:, 0:1]) * normal[:, 0:1]
    return np.imag(-g)

def func0_outer(x, y):
    normal = outer.boundary_normal(x)
    result = - k0 * y[:, 1:2]
    return result

def func1_outer(x, y):
    normal = outer.boundary_normal(x)
    result =  k0 * y[:, 0:1]
    return result


    
bc0_inner = dde.NeumannBC(geom, func0_inner, boundary_inner, component = 0)
bc1_inner = dde.NeumannBC(geom, func1_inner, boundary_inner, component = 1)

bc0_outer = dde.RobinBC(geom, func0_outer, boundary_outer, component = 0)
bc1_outer = dde.RobinBC(geom, func1_outer, boundary_outer, component = 1)


bcs = [bc0_inner, bc1_inner, bc0_outer, bc1_outer]
#weights = [1, 1, 100, 100, 100, 100]
weights = [1, 1, 1, 1, 1, 1]


data = dde.data.PDE(geom, pde, bcs, num_domain= nx**2, num_boundary= 8 * nx, num_test= 5 * nx ** 2, solution = sol)
net = dde.maps.FNN([2] + [50] * 4 + [2], "tanh", "Glorot uniform")
model = dde.Model(data, net)

model.compile("adam", lr=0.0075, loss_weights= weights, metrics=["l2 relative error"])
losshistory, train_state = model.train(epochs= 20000)

dde.saveplot(losshistory, train_state, issave=True, isplot=True)

#model.compile("L-BFGS")
#losshistory, train_state = model.train()

Compiling model...
Building feed-forward neural network...
'build' took 0.041358 s



/opt/conda/envs/DL/lib/python3.9/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/opt/conda/envs/DL/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2021-10-29 09:53:32.542386: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect StreamExecutor with strength 1 edge matrix:
2021-10-29 09:53:32.542438: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1264]      


'compile' took 0.724805 s

Initializing variables...
Training model...

Step      Train loss                                                      Test loss                                                       Test metric   
0         [7.55e+00, 4.91e-01, 4.89e-01, 1.57e+00, 1.92e-01, 2.66e+00]    [7.83e+00, 5.14e-01, 4.89e-01, 1.57e+00, 1.92e-01, 2.66e+00]    [1.81e+00]    
1000      [3.34e-02, 5.77e-02, 8.79e-03, 6.84e-03, 2.13e-03, 3.16e-03]    [4.56e-02, 9.18e-02, 8.79e-03, 6.84e-03, 2.13e-03, 3.16e-03]    [8.55e-01]    
2000      [1.69e-02, 3.06e-02, 1.81e-03, 1.66e-03, 1.46e-03, 2.64e-03]    [4.15e-02, 1.18e-01, 1.81e-03, 1.66e-03, 1.46e-03, 2.64e-03]    [7.13e-01]    
3000      [7.43e-03, 1.25e-02, 4.51e-04, 6.29e-04, 4.36e-04, 9.60e-04]    [9.53e-02, 2.00e-01, 4.51e-04, 6.29e-04, 4.36e-04, 9.60e-04]    [6.59e-01]    
4000      [5.38e-03, 5.40e-03, 2.56e-04, 4.14e-04, 1.88e-04, 5.26e-04]    [1.90e-01, 3.04e-01, 2.56e-04, 4.14e-04, 1.88e-04, 5.26e-04]    [6.63e-01]    


In [ ]:

'''            Evaluate field over a specified grid of points              '''
# Square grid with 10 points per wavelength in each direction
Nx = int(np.ceil(dim_x/wave_len * 30))
Ny = Nx


# Grid points
xmin, xmax, ymin, ymax = [-dim_x/2, dim_x/2., -dim_x/2, dim_x/2]
plot_grid = np.mgrid[xmin:xmax:Nx * 1j, ymin:ymax:Ny * 1j]
points = np.vstack((plot_grid[0].ravel(),
                    plot_grid[1].ravel(),
                    np.zeros(plot_grid[0].size)))

points_2d = points[:2, :]

#in_circ = points[0, :]**2 + points[1, :]**2 <= (radius)**2
#in_circ_2d = points_2d[0, :]**2 + points_2d[1, :]**2 <= (radius)**2
#points[0, in_circ] = -radius - wave_len / 10
#points[1, in_circ] = radius + wave_len / 10
#points[2, in_circ] = 0.

# Bounding box tree etc for function evaluations

u_sca = model.predict(points[:2, :].T)
u_sca = u_sca[:, 0] + 1j* u_sca[:, 1]

u_exact = sol(points_2d.T)
u_exact = u_exact[:, 0] + 1j * u_exact[:, 1]

ide = np.sqrt(points_2d[0, :] ** 2 + points_2d[1, :] ** 2) <= R

u_exact[ide] = 0
u_sca[ide] = 0

u_exact = u_exact.reshape((Nx, Ny))
u_sca = u_sca.reshape((Nx, Ny))

'''                  Compare against analytical solution                    '''
# Uncomment to perform comparison, takes a few seconds to run



diff = u_exact-u_sca
error = np.linalg.norm(diff)/np.linalg.norm(u_exact)
print('Relative error = ', error)

'''                     Plot field and save figure                          '''
plt.rc('font', family='serif', size=22)



fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize = (24,12))

pcm = ax1.imshow(np.fliplr(np.real(u_sca)).T,
           extent=[-dim_x/2, dim_x/2., -dim_x/2, dim_x/2],
           cmap=plt.cm.get_cmap('seismic'), interpolation='spline16', label='PINN')

circle = plt.Circle((0., 0.), R, color='black', fill=True)
ax1.add_patch(circle)
#ax1.axis('off')
fig.colorbar(pcm, ax = ax1)

pcm = ax2.imshow(np.fliplr(np.real(u_exact)).T,
           extent=[-dim_x/2, dim_x/2., -dim_x/2, dim_x/2],
           cmap=plt.cm.get_cmap('seismic'), interpolation='spline16', label = 'Exact')

circle = plt.Circle((0., 0.), R, color='black', fill=True)
ax2.add_patch(circle)

#ax1.axis('off')
ax1.set_title('PINNs')
ax2.set_title('Exact')
fig.colorbar(pcm, ax = ax2)


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize = (24,12))

pcm = ax1.imshow(np.fliplr(np.imag(u_sca)).T,
           extent=[-dim_x/2,dim_x/2.,-dim_x/2,dim_x/2],
           cmap=plt.cm.get_cmap('seismic'), interpolation='spline16', label='PINN')


ax1.axis('off')
circle = plt.Circle((0., 0.), R, color='black', fill=True)
ax1.add_patch(circle)
fig.colorbar(pcm, ax = ax1)

pcm = ax2.imshow(np.fliplr(np.imag(u_exact)).T,
           extent = [-dim_x/2,dim_x/2.,-dim_x/2,dim_x/2],
           cmap=plt.cm.get_cmap('seismic'), interpolation='spline16', label = 'Exact')

circle = plt.Circle((0., 0.), R, color='black', fill=True)
ax2.add_patch(circle)


#ax1.axis('off')
ax1.set_title('PINNs')
ax2.set_title('Exact')
fig.colorbar(pcm, ax = ax2)
